In [128]:
import math
import numpy as np
import matplotlib.pyplot as plt

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img,sigma=0.33):
    """Applies the Canny transform"""
    new_img = np.median(img)
    lower = int(max(0, (1.0 - sigma) * new_img))
    upper = int(min(255, (1.0 + sigma) * new_img))
    return cv2.Canny(img, lower, upper)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    `vertices` should be a numpy array of integer points.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    #cv2.fillPoly(mask, vertices, ignore_mask_color)
    cv2.fillPoly(mask, np.array([vertices], dtype=np.int32),ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[0, 0, 255], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    x=[]
    y=[]
    for line in lines:
        for x1,y1,x2,y2 in line:
            x += [x1,x2]
            y += [y1,y2]
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)
        z = np.polyfit(x,y,1)
        f = np.poly1d(z)
        #cv2.line(img,(0,max(x.astype(int))),(f(0),f(max(x.astype(int)))),'r')
        #cv2.line(img,(0, max(x).astype(int)), (f(0).astype(int), f(max(x)).astype(int)),color,thickness)

        #plt.axis([0, 60, 0, 60])
        plt.show()
        
        print(line)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + γ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)

#def extrapolate_img(img,)

In [129]:
import numpy as np
theta = (np.pi/180) * 180
print(theta)

3.141592653589793


In [130]:
#importing some useful packages
import os
import math
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline
# TODO: Build your pipeline that will draw lane lines on the test_image
# then save them to the test_images_output directory.
def ProcessingImage(image,image_name):
    ROI_Ratio = 1.8
    height = np.size(image, 0)
    width = np.size(image, 1)
    diagonal = int(math.sqrt(height*height + width*width))
    print(width,height)
    #Rectangle ROI
    #vertices = np.array( [[0,height/ROI_Ratio],[0,height],[width,height],[width,height/ROI_Ratio]], dtype=np.int32 )
    #Pyramid ROI
    vertices = np.array( [[0 + width*0.06,height],[width/2,(height/2)+height*0.05],[width-width*0.06,height]], dtype=np.int32 )

    gray = grayscale(image)
    blur = gaussian_blur(gray,3)
    can = canny(blur,0.33)
    roi = region_of_interest(can,vertices)
    rho = 1
    theta = (np.pi/180) #One Radian
    threshold = 10
    min_line_length = 2
    max_line_gap = 1
    #rho_arr = [1,15,30,45,60,75,90,115,130,145,150,165,180]
    #thresholds = [10, 50, 90]
    #for max_line_gap in range(1,10,2):
        #for min_line_length in range(1,16,3):
    hl = hough_lines(roi,rho,theta,threshold,min_line_length,max_line_gap)
    #dump_image = "./dump/" + 'max_' + str(max_line_gap)+ '_minLen_' + str(min_line_length) + '.jpg'
    #dump_image = "./dump/out.jpg"
    #dump_image = "./" + image_name
    #cv2.imwrite(dump_image,hl)
    
    wei = weighted_img(image,hl,0.8,1,0)
    dump_image = "./dump/" + image_name
    cv2.imwrite(dump_image,wei)
    
    return image

#single image
single_path = "test_images/solidWhiteCurve.jpg"
image_name = "solidWhiteCurve_theta_90.jpg"
#ProcessingImage(cv2.imread(single_path),image_name)

#pipeline of images
multipath = "test_images/"
for f in os.listdir(multipath):
    image = cv2.imread(multipath+f)
    ProcessingImage(image,f)
    
    



960 540
[[509 326 641 402]]
[[538 342 566 358]]
[[623 386 898 538]]
[[602 379 640 401]]
[[483 304 485 305]]
[[736 456 844 520]]
[[648 405 736 457]]
[[490 305 492 304]]
[[306 440 345 410]]
[[389 382 391 380]]
[[292 462 353 411]]
[[509 322 513 325]]
[[485 308 489 311]]
[[280 460 284 457]]
[[512 325 564 353]]
[[639 400 655 410]]
[[453 330 461 323]]
[[842 518 876 538]]
[[453 329 461 322]]
[[565 355 630 390]]
[[286 457 288 455]]
[[451 335 466 323]]
[[409 366 420 357]]
[[286 456 317 432]]
[[280 461 285 457]]
[[542 342 580 362]]
[[631 391 645 399]]
[[489 319 493 319]]
[[338 425 341 422]]
[[414 358 418 355]]
[[296 449 301 445]]
[[407 363 412 358]]
[[484 309 486 308]]
[[295 461 304 453]]
[[480 320 488 320]]
[[339 414 344 410]]
[[440 320 446 320]]
[[456 329 458 327]]
[[724 449 733 455]]
[[461 325 463 323]]
[[529 335 533 337]]
[[351 414 353 412]]
[[459 328 463 325]]
[[495 316 498 317]]
[[387 382 389 380]]
[[481 311 484 312]]
[[291 453 293 451]]
[[449 326 451 324]]
[[797 483 898 539]]
[[494 320 49